In [1]:
import findspark
findspark.init() 
from pyspark.sql import functions as F
from pyspark.sql import Window
from pyspark.sql.types import TimestampType
import time

In [2]:
from pyspark.sql import SparkSession
# starting SparkSession
spark = SparkSession \
        .builder \
        .appName("bg_sql_csv") \
        .config("spark.ui.port","4041") \
        .config("spark.executor.memory", "2g") \
        .config("spark.executor.cores", "2") \
        .config("spark.task.maxFailures", "8") \
        .getOrCreate()

# Read csvs files with inferschema variable

In [3]:
# Read artist.csv
df_artists = spark.read.options(inferSchema='True',delimiter=',') \
  .csv("hdfs://master:9000/files/artists.csv")

df_artists = df_artists.withColumnRenamed('_c0','artist_id')\
                       .withColumnRenamed('_c1','artist_name')

# Read chart_artist_mapping.csv
df_chart_artist_mapping = spark.read.options(inferSchema='True',delimiter=',') \
  .csv("hdfs://master:9000/files/chart_artist_mapping.csv")

df_chart_artist_mapping = df_chart_artist_mapping.withColumnRenamed('_c0','song_id')\
                                                 .withColumnRenamed('_c1','artist_id')

# Read charts.csv
df_charts = spark.read.options(inferSchema='True',delimiter=',') \
  .csv("hdfs://master:9000/files/charts.csv")

df_charts = df_charts.withColumnRenamed('_c0','song_id')\
                     .withColumnRenamed('_c1','song_title')\
                     .withColumnRenamed('_c2','rank')\
                     .withColumnRenamed('_c3','date')\
                     .withColumnRenamed('_c4','region_id')\
                     .withColumnRenamed('_c5','chart')\
                     .withColumnRenamed('_c6','chart_traffic')\
                     .withColumnRenamed('_c7','streams')\
                     .withColumn('date',F.col('date')+F.expr('INTERVAL 3 HOURS'))\
                     .withColumn('year',F.year(F.col('date')))\
                     .withColumn('month',F.month(F.col('date')))\
                     .withColumn('day',F.dayofmonth(F.col('date')))\
                     .withColumn('only_date',F.to_date(F.col('date')))

# Read regions.csv
df_regions = spark.read.options(inferSchema='True',delimiter=',') \
  .csv("hdfs://master:9000/files/regions.csv")

df_regions = df_regions.withColumnRenamed('_c0','region_id')\
                       .withColumnRenamed('_c1','region_name')


# Extract parquet files based on dataframes

In [4]:
# df_artists.write.parquet("hdfs://master:9000/files/artists.parquet")
# df_chart_artist_mapping.write.parquet("hdfs://master:9000/files/chart_artist_mapping.parquet")
# df_charts.write.parquet("hdfs://master:9000/files/charts.parquet")
df_regions.write.parquet("hdfs://master:9000/files/regions.parquet")

# Method SparkSQL with CSV file as input file

## Create Temporary Views

In [5]:
df_charts.createOrReplaceTempView("charts")
df_regions.createOrReplaceTempView("regions")
df_chart_artist_mapping.createOrReplaceTempView("chart_artist_mapping")
df_artists.createOrReplaceTempView("artists")

## Q1: Ποιο είναι το συνολικό πλήθος των streams που έχουν καταγραφεί για το τραγούδι με τίτλο “Shape of You”, σύμφωνα με τα top200 charts?
Ως αποτελέσμα να δωθεί μόνο ένας αριθμός με το πλήθος

In [ ]:
q1 = spark.sql('''SELECT sum(streams) 
                  FROM charts 
                  WHERE song_title = "Shape of You" and chart = "top200"''')

In [ ]:
start_time = time.time()
q1.coalesce(1).write.option("header","true").option("sep",",").mode("overwrite").csv("hdfs://master:9000/outputs/sql_q1_csv.csv")
print("Execution Time for q1(csv) is: ",time.time() - start_time)

## Q2: Για κάθε chart, να βρεθεί το τραγούδι με τον μεγαλύτερο μέσο χρόνο παραμονής (δείτε «Υποδείξεις») στην πρώτη θέση.
Ως αποτέλεσμα, αναμένονται δύο γραμμές, μία για κάθε chart στην μορφή:
όνομα_chart, όνομα_τραγουδιού, μέσος_χρόνος_παραμονής_θέση#1
Αναμενόμενο αποτέλεσμα στο viral50 chart viral50,Calma - Remix,24.985507

In [ ]:
q2 = spark.sql('''select chart, song_title, avg_cnt from (select chart, song_title, t.avg_cnt, max(t.avg_cnt) over (partition by t.chart) as max_r
                    from (select chart,song_title,count(*)/69 as avg_cnt
                          from charts
                          where rank = 1
                          group by chart, song_title)t)t1
                    where avg_cnt = max_r
                    ''')

In [ ]:
start_time = time.time()
q2.coalesce(1).write.option("header","true").option("sep",",").mode("overwrite").csv("hdfs://master:9000/outputs/sql_q2_csv.csv")
print("Execution Time for q2(csv) is: ",time.time() - start_time)

## Q3: Από τα top200 charts, να βρεθεί για κάθε μήνα της κάθε χρονιάς, το μέσο ημερήσιο πλήθος streams του τραγουδιού που βρίσκεται στην θέση 1 (δείτε «Υποδείξεις»), ταξινομημένα ως προς την χρονιά και τον μήνα.

In [ ]:
q3 = spark.sql('''SELECT year, month, avg_daily_sum 
                  FROM (SELECT year, month, sum(streams)/count(distinct day) as avg_daily_sum
                  FROM charts 
                  WHERE rank = 1 and chart = "top200"
                  GROUP BY year, month
                  ORDER BY year, month asc) t
                  ''')

In [ ]:
start_time = time.time()
q3.coalesce(1).write.option("header","true").option("sep",",").mode("overwrite").csv("hdfs://master:9000/outputs/sql_q3_csv.csv")
print("Execution Time for q3(csv) is: ",time.time() - start_time)

## Q4: Από τα viral50 charts, βρείτε για κάθε χώρα το (ή τα σε περίπτωση ισοψηφίας) τραγούδια με το μεγαλύτερο πλήθος παραμονής στο charts. Ταξινομείστε τα αποτελέσματα σας ως προς το όνομα της χώρας και το όνομα του τραγουδιού. Ως αποτέλεσμα δώστε μία γραμμή για κάθε τραγούδι κάθε χώρας στην μορφή :
χώρα, id_τραγουδιού, όνομα_τραγουδιού, πλήθος_παραμονής_στο_viral50

In [ ]:
q4 = spark.sql('''select regions.region_name, t2.song_id, t2.song_title, t2.cnt
                  from regions
                  inner join (SELECT region_id, song_id, song_title,cnt FROM (SELECT region_id, song_id,song_title, t.cnt,max(t.cnt) over (partition by t.region_id) as max_r FROM 
                  (SELECT region_id, song_id,song_title, count(*) as cnt
                  FROM charts
                  WHERE chart = "viral50"
                  GROUP BY region_id, song_id, song_title) t
                  order by t.cnt desc) t1
                  where t1.cnt = t1.max_r) t2
                  on regions.region_id = t2.region_id
                  order By regions.region_name,  t2.song_title''')

In [ ]:
start_time = time.time()
q4.coalesce(1).write.option("header","true").option("sep",",").mode("overwrite").csv("hdfs://master:9000/outputs/sql_q4_csv.csv")
print("Execution Time for q4(csv) is: ",time.time() - start_time)

## Q5:Σύμφωνα με τα top200, βρείτε σε κάθε χρονιά τον καλλιτέχνη με το μεγαλύτερο μέσο πλήθος streams. Ταξινομείστε ως προς τη χρονιά.
Ως αποτελέσμα, δώστε για κάθε χρονιά μία γραμμή στην εξής μορφή
χρονιά, όνομα_καλλιτέχνη, μέσο_πλήθος_streams

In [ ]:
q5 = spark.sql('''select year, artist_name, total_streams as avg_streams 
                  from(select year, artist_name, total_streams, max(total_streams) over (partition by year) as max_streams
                       from (select year, artist_name, sum(streams)/69 as total_streams 
                             from(select * 
                                  from (select * 
                                        from (select * 
                                              from charts
                                              where chart = "top200") t
                                              left join chart_artist_mapping
                                              on t.song_id = chart_artist_mapping.song_id) t1
                                        inner join artists
                                        on t1.artist_id = artists.artist_id ) t2 
                                  group by year, artist_name) t3) t4   
                  where total_streams = max_streams
                  order by year
                  ''')



In [ ]:
start_time = time.time()
q5.coalesce(1).write.option("header","true").option("sep",",").mode("overwrite").csv("hdfs://master:9000/outputs/sql_q5_csv.csv")
print("Execution Time for q5(csv) is: ",time.time() - start_time)

## Q6: Για την Ελλάδα, βρείτε για κάθε χρονιά και chart τον καλλιτέχνη (ή τους καλλιτέχνες) που έχει (έχουν) παραμείνει διαδοχικές ημέρες περισσότερες φορές στο #1 κάποιο από τα τραγούδια του. Ταξινομείστε ως προς το chart και τη χρονιά.
Ως αποτελέσμα δώστε μία γραμμή για κάθε καλλιτέχνη κάθε χρονιάς στη μορφή: όνομα_chart, χρονιά, όνομα καλλιτέχνη, ημέρες_διαδοχικής_παραμονής_#1

In [6]:
q6 = spark.sql('''With table_1 as(select chart, year, song_id, max_streak
                     from (select chart, year, song_id, sum(cnt) as streak, max(sum(cnt)) over (partition by chart,year) as max_streak
                            from (select chart, year, song_id ,date_group, count(*) as cnt,min(date),max(date) 
                                  from (select chart, year, song_id, date, date_add(date, - rank() over (partition by chart, year, song_id order by date)) as date_group
                                        from (select chart, year, song_id, only_date as date
                                              from charts
                                              where region_id = 23 and chart_traffic = "SAME_POSITION" and rank = 1) in_t ) t
                            group by chart, year, song_id,date_group) t1
                            group by chart, year, song_id) t3
                     where streak = max_streak),
                     table_2 as (select song_id, artists.artist_name 
                                from (select charts.song_id as song_id, chart_artist_mapping.artist_id as artist_id 
                                      from charts
                                      left join chart_artist_mapping
                                      on charts.song_id = chart_artist_mapping.song_id) t1
                                inner join artists
                                on t1.artist_id = artists.artist_id) 
                      select Distinct chart, year, artist_name, max_streak
                      from table_1
                      inner join table_2 
                      on table_1.song_id = table_2.song_id
                      order by chart desc, year 
                     ''')

In [7]:
start_time = time.time()
q6.coalesce(1).write.option("header","true").option("sep",",").mode("overwrite").csv("hdfs://master:9000/outputs/sql_q6_csv.csv")
print("Execution Time for q6(csv) is: ",time.time() - start_time)

Execution Time for q6(csv) is:  61.98225927352905


In [8]:
spark.stop()